In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ml-w13p1/train.csv
/kaggle/input/2022-ml-w13p1/test.csv
/kaggle/input/2022-ml-w13p1/sample_submit.csv


In [2]:
submit = pd.read_csv('../input/2022-ml-w13p1/sample_submit.csv')
test = pd.read_csv('../input/2022-ml-w13p1/test.csv')
train = pd.read_csv('../input/2022-ml-w13p1/train.csv')

In [3]:
train

,index,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
3,4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
4,5,21651,city_176,0.764,NaN,Has relevent experience,Part time course,Graduate,STEM,11,NaN,NaN,1,24,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15321,19152,29754,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,Humanities,7,10/49,Funded Startup,1,25,0.0
15322,19153,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42,1.0
15323,19154,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52,1.0
15324,19156,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,0.0


In [4]:
x_train = train.drop(['index', 'enrollee_id', 'city', 'target'], axis=1)
y_train = train['target']
x_test = test.drop(['index', 'enrollee_id', 'city'], axis=1)

In [5]:
# object : 범주형 데이터 -> label encoder를 통해 수치형으로 변환시켜줘야함
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15326 entries, 0 to 15325
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city_development_index  15326 non-null  float64
 1   gender                  11752 non-null  object 
 2   relevent_experience     15326 non-null  object 
 3   enrolled_university     15009 non-null  object 
 4   education_level         14964 non-null  object 
 5   major_discipline        13068 non-null  object 
 6   experience              15276 non-null  object 
 7   company_size            10612 non-null  object 
 8   company_type            10445 non-null  object 
 9   last_new_job            14983 non-null  object 
 10  training_hours          15326 non-null  int64  
dtypes: float64(1), int64(1), object(9)
memory usage: 1.3+ MB


In [6]:
# train + test 를 label encoder에 집어 넣기위해 병합
x_data = pd.concat([x_train, x_test], axis=0)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# 라벨 인코딩은 열마다 진핼시켜줘야 한다. 
# 원래 빼왔던 곳에 다시 넣어주자 !

le.fit(x_data['gender'])
x_train['gender'] = le.fit_transform(x_train['gender'])
x_test['gender'] = le.fit_transform(x_test['gender'])

le.fit(x_data['relevent_experience'])
x_train['relevent_experience'] = le.fit_transform(x_train['relevent_experience'])
x_test['relevent_experience'] = le.fit_transform(x_test['relevent_experience'])

le.fit(x_data['enrolled_university'])
x_train['enrolled_university'] = le.fit_transform(x_train['enrolled_university'])
x_test['enrolled_university'] = le.fit_transform(x_test['enrolled_university'])

le.fit(x_data['education_level'])
x_train['education_level'] = le.fit_transform(x_train['education_level'])
x_test['education_level'] = le.fit_transform(x_test['education_level'])

le.fit(x_data['major_discipline'])
x_train['major_discipline'] = le.fit_transform(x_train['major_discipline'])
x_test['major_discipline'] = le.fit_transform(x_test['major_discipline'])

le.fit(x_data['company_type'])
x_train['company_type'] = le.fit_transform(x_train['company_type'])
x_test['company_type'] = le.fit_transform(x_test['company_type'])

# experience, company_size, last_new_job 는 빼고 실행해 줬다. 

In [7]:
# 해당 열의 모든 정보를 확인 가능 ! x_train_le 하면 노노~
x_test['experience'].unique()

array(['5', '17', '2', '16', '11', '14', '>20', '9', '4', '6', '12', '8',
       '15', '13', '1', '3', '<1', '10', '7', '18', nan, '20', '19'],
      dtype=object)

In [8]:
x_train.loc[x_train.experience == '>20', 'experience'] = 21
x_train.loc[x_train.experience == '<1','experience'] = 0 
x_train.loc[x_train.experience.isnull(), 'experience'] = -1
x_train['experience'] = x_train['experience'].astype('int64')

In [9]:
# temp [조건문이 == true 일 떄] = 이 값으로 변경 해주는 것.

temp = x_test['experience'].copy()
temp[temp == '>20'] =5
temp[temp == '<1'] =5
temp[temp.isnull()] = -1
x_test['experience'] = temp.astype('int64')

In [10]:
x_train['company_size'].unique()

array([nan, '50-99', '<10', '1000-4999', '10000+', '100-500', '5000-9999',
       '10/49', '500-999'], dtype=object)

In [11]:
x_train.loc[x_train.company_size == '10000+', 'company_size'] = 1
x_train.loc[x_train.company_size == '5000-9999', 'company_size'] = 2
x_train.loc[x_train.company_size == '1000-4999', 'company_size'] = 3
x_train.loc[x_train.company_size == '500-999', 'company_size'] = 4
x_train.loc[x_train.company_size == '100-500', 'company_size'] = 5
x_train.loc[x_train.company_size == '50-99', 'company_size'] = 6
x_train.loc[x_train.company_size == '10/49', 'company_size'] = 7
x_train.loc[x_train.company_size == '<10', 'company_size'] = 8
x_train.loc[x_train.company_size.isnull(), 'company_size'] = -1
x_train['company_size'] = x_train['company_size'].astype('int64')

In [12]:
temp = x_test['company_size'].copy()
temp[temp == '10000+'] = 1
temp[temp == '5000-9999'] = 2
temp[temp == '1000-4999'] = 3
temp[temp == '500-999'] = 4
temp[temp == '100-500'] = 5
temp[temp == '50-99'] = 6
temp[temp == '10/49'] = 7
temp[temp == '<10'] = 8
temp[temp.isnull()] = -1
x_test['company_size'] = temp.astype('int64')

In [13]:
x_train['last_new_job'].unique() 

array(['1', '>4', 'never', '4', '3', '2', nan], dtype=object)

In [14]:
x_train.loc[x_train.last_new_job == '>4', 'last_new_job'] = 5
x_train.loc[x_train.last_new_job == 'never', 'last_new_job'] = 0
x_train.loc[x_train.last_new_job.isnull() , 'last_new_job'] = -1
x_train['last_new_job'] = x_train['last_new_job'].astype('int64')

In [15]:
temp = x_test['last_new_job'].copy()
temp[temp == '>4'] =5
temp[temp == 'never'] =5
temp[temp.isnull()] = -1
x_test['last_new_job'] = temp.astype('int64')

In [16]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

qda = QuadraticDiscriminantAnalysis()
qda.fit(x_train, y_train)
submit['target'] = qda.predict(x_test).astype('int64')

In [17]:
submit.to_csv('result.csv', index=False)